In [27]:
# Import of basic libraries
import tensorflow as tf
from glob import glob
import pandas as pd
import cv2
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from builtins import range, input
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing import image

In [28]:
# Load saved model
model = load_model('densenet201_ct.h5')

In [29]:
# To run the next few cells create a folder with the 'X_train' images before the preprocess
import random

img_paths = glob(".\\kaggle\\val_set_kaggle\\*.png")
img_paths = [(img_paths[index]) for index in random.sample(range(0, len(img_paths)), len(img_paths))]

covid = 0
non_covid = 0

for file in img_paths:
    if "Non-Covid" in file:
        non_covid += 1
    else:
        covid += 1

In [30]:
val_imgs = []

for img_path in img_paths:
    img = cv2.imread(img_path)                 # read file 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # arrange format as per kera
    img = cv2.resize(img,(224,224))            # resize as per model
    val_imgs.append(img)  

In [31]:
# Load the model that was trained above
model = keras.models.load_model("densenet201_ct.h5")

last_conv_layer = "conv5_block32_concat"

def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size) # `img` is a PIL image of size 224x224
    array = keras.preprocessing.image.img_to_array(img)                  # `array` is a float32 Numpy array of shape (224, 224, 3)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 224, 224, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted)
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap_1 = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap_1)

    # For visualization purpose, also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [32]:
# Remove last layer's softmax
model.layers[-1].activation = None

In [33]:
# Display
from IPython.display import Image, display
import matplotlib.cm as cm

def create_gradcam(img_br, heatmap, cam_path="cam.jpg", alpha=0.3):
    heatmap = np.uint8(255 * heatmap)        # rescale heatmap to a range 0-255

    jet = cm.get_cmap("jet")                 # use jet colormap to colorize heatmap

    jet_colors = jet(np.arange(256))[:, :3]  # use RGB values of the colormap
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_br.shape[1], img_br.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    superimposed_img_rgb = keras.preprocessing.image.array_to_img(superimposed_img)
    
    return superimposed_img_rgb

In [34]:
# Use this snippet of code to apply grad-cam and create a folder with those images
import shutil

for i, img in enumerate(val_imgs):
    grad_cam = make_gradcam_heatmap(np.expand_dims(img, axis=0), model, last_conv_layer)
    grad_cam_emphasized = create_gradcam(img, grad_cam, cam_path="cam.jpg", alpha=0.3)
    
    file_destination_path = os.path.join('.\\deep\\kaggle\\grad_cam_images', os.path.splitext(os.path.basename(img_paths[i]))[0])
    grad_cam_emphasized.save(file_destination_path + ".png")

In [ ]:
import shutil

# Visualize five random images
plt.figure(figsize=(8, 8))

for i, img in enumerate(val_imgs[:5], 1):
    ax = plt.subplot(1, 5, i)
    grad_cam = make_gradcam_heatmap(np.expand_dims(img, axis=0), model, last_conv_layer)
    grad_cam_emphasized = create_gradcam(img, grad_cam, cam_path="cam.jpg", alpha=0.4)
    plt.axis('off')
    plt.imshow(grad_cam_emphasized)
plt.tight_layout()